In [1]:
#%matplotlib inline
import pickle
import copy
import numpy as np
import random
import os
import torch
os.chdir('..')
from test_utils import *
from ipywidgets import interact


# 데이터 직접 입력

In [2]:
import argparse
os.chdir('..')
from lib.utils.learning import *
from lib.utils.tools import *

In [3]:
from test_utils import *

h36m_connections = [
    (0,1),
    (0,4),
    (0,7),
    (1,2),
    (2,3),
    (4,5),
    (5,6),
    (7,8),
    (8,9),
    (9,10),
    (8,14),
    (14,15),
    (15,16),
    (8,11),
    (11,12),
    (12,13)
]

In [4]:
def draw_skeleton_2d(points, connections, elevation=0, azimuth=0):
    fig = go.Figure()
    
    # Add points
    for point in points.values():
        fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]],
                                   mode='markers', marker=dict(size=2, color='blue')))

    # Add connections
    for connection in connections:
        p1, p2 = points[connection[0]], points[connection[1]]
        fig.add_trace(go.Scatter(x=[p1[0], p2[0]], y=[p1[1], p2[1]],
                                   mode='lines', line=dict(color='black')))

    img_width = 1920
    img_height = 1080
    scale_factor = 0.5

    # https://wikidocs.net/185955 참고
    fig.update_xaxes(range=[-1, 1])
    fig.update_yaxes(range=[1, -1])
    #fig.update_xaxes(autorange="reversed") # 축 범위 반전, 위의 range 설정이 초기화되버림

    fig.update_layout(width=700,height=700)
    
    fig.update_layout(template="plotly_white")

    fig.show()

In [33]:
# 예제 CLIP 파일
os.chdir('/home/hrai/codes/MotionBERT')
with open('data/motion3d/MB3D_f243s81/AIHUB_30_243/train/00000000.pkl', 'rb') as f:
	test = pickle.load(f)
frame_num = 150
batch_input = torch.tensor(test['data_input'][frame_num].reshape(1, 1, 17, 3))

In [34]:
# 2d input
aihub_2d = copy.deepcopy(test['data_input'][frame_num])
aihub_2d.shape

(17, 3)

In [35]:
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.gridspec as gridspec
from test_utils import *

plt.switch_backend('TkAgg')
matplotlib.rcParams['pdf.fonttype'] = 42

matplotlib.rcParams['ps.fonttype'] = 42

In [39]:
aihub_2d

array([[-0.05533767,  0.01085418,  0.9175866 ],
       [-0.0282281 ,  0.02169806,  0.84879994],
       [ 0.08021009,  0.07049525,  0.91875553],
       [ 0.02056909,  0.17351145,  0.9085786 ],
       [-0.08244717,  0.01085418,  0.8794954 ],
       [-0.03365004,  0.03796381,  0.8484117 ],
       [-0.07702529,  0.13555813,  0.9441897 ],
       [-0.03636092, -0.08402914,  0.93751   ],
       [-0.01738429, -0.17891252,  0.9574334 ],
       [ 0.02599096, -0.19517824,  0.86836433],
       [-0.00111854, -0.24397543,  0.9137768 ],
       [-0.06075954, -0.15180299,  0.9573031 ],
       [-0.03907192, -0.09216201,  0.83274007],
       [ 0.03141284, -0.1084277 ,  0.8711533 ],
       [ 0.01514709, -0.14095917,  0.9097614 ],
       [ 0.09105384, -0.10300583,  0.89516866],
       [ 0.06394434, -0.11384964,  0.8534446 ]], dtype=float32)

In [37]:
result = get_2d_pose_image(aihub_2d*1000, np.zeros([1000, 1000, 3]), dataset='h36m')
plot_cv2_image(result)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [7]:
# 2d input
aihub_2d = copy.deepcopy(test['data_input'][frame_num])
# 2d skeleton
pos = copy.deepcopy(aihub_2d)
points_aihub_2d = {}
for i, p in enumerate(pos):
    p = np.append(p, 0.0)
    #print(i, h36m_keypoints[i], p)
    points_aihub_2d[i] = p
pos_vector = pos.reshape(-1) # (51,)
total_min, total_max = pos_vector.min(), pos_vector.max()
print("total min: ", total_min,
      "total max: ", total_max)

interact(draw_skeleton_2d(points_aihub_2d, h36m_connections))

total min:  -0.2416828155517578 total max:  0.28986778259277335


In [12]:
# gt
aihub_3d = copy.deepcopy(test['data_label'][frame_num])
pos = copy.deepcopy(aihub_3d)
points_aihub_3d = {}
for i, p in enumerate(pos):
    #print(i, h36m_keypoints[i], p)
    points_aihub_3d[i] = np.array([p[0], -p[2], -p[1]]) #np.array([p[0], p[1], p[2]])  #np.array([p[0], -p[2], p[1]])
pos_vector = pos.reshape(-1) # (51,)
total_min, total_max = pos_vector.min(), pos_vector.max()
print("total min: ", total_min,
      "total max: ", total_max)
fig = draw_skeleton(points_aihub_3d, h36m_connections)
fig.show()

total min:  -0.87315625 total max:  0.34732916666666663
